In [ ]:
import chipwhisperer as cw
scope = cw.scope()
print("done")

In [ ]:
target = cw.target(scope, cw.targets.SimpleSerial) #cw.targets.SimpleSerial can be omitted


In [ ]:
scope.default_setup()


In [ ]:
%%bash
cd ../firmware/mcu/simpleserial-base/
make PLATFORM=CWLITEXMEGA CRYPTO_TARGET=NONE clean
make PLATFORM=CWLITEXMEGA CRYPTO_TARGET=NONE

In [ ]:
# Lower level programming attempt
prog = cw.programmers.XMEGAProgrammer()
prog.scope = scope

try:
    print("Manual programming attempt...")
    
    # Set up pins manually before opening programmer
    scope.io.pdic = 'high_z'
    scope.io.pdid = 'high_z'
    scope.io.nrst = 'low'
    time.sleep(0.2)
    
    scope.io.nrst = 'high_z'
    time.sleep(0.1)
    
    # Open programmer
    prog.open()
    
    # Try to find target multiple times
    found = False
    for attempt in range(3):
        try:
            print(f"Find attempt {attempt + 1}...")
            prog.find()
            found = True
            break
        except:
            time.sleep(0.2)
    
    if not found:
        raise Exception("Could not find target after 3 attempts")
    
    # Skip erase and try direct programming
    print("Attempting direct programming (skipping erase)...")
    prog.program("../firmware/mcu/simpleserial-base/simpleserial-base-CWLITEXMEGA.hex", 
                memtype="flash", verify=False)
    
    print("Programming successful!")
    
except Exception as e:
    print(f"Programming failed: {e}")
finally:
    try:
        prog.close()
    except:
        pass

In [ ]:
# Reset target and test communication
import time

scope.io.nrst = 'low'
time.sleep(0.1)
scope.io.nrst = 'high_z'
time.sleep(0.5)  # Give target time to boot

print("Testing target communication...")

In [ ]:
# Test the 'p' command (should echo back)
msg = bytearray([0]*16)
target.simpleserial_write('p', msg)
response = target.simpleserial_read('r', 16)
print(f"Response to 'p' command: {response}")

In [ ]:
# Clear serial buffers
target.flush()
print("Buffers cleared")

In [ ]:
# Try a lower baud rate
target.baud = 38400  # Default is usually 115200
print(f"Baud rate set to: {target.baud}")

In [ ]:
# Test with cleared buffers and new baud rate
import time

target.flush()
time.sleep(0.1)

msg = bytearray([0]*16)
target.simpleserial_write('p', msg)
response = target.simpleserial_read('r', 16)
print(f"Response to 'p' command: {response}")

In [ ]:
target.simpleserial_write('k', msg)


In [ ]:
print(target.simpleserial_wait_ack()) #should return 0


In [ ]:
target.write('p' + "0Z"*16 + '\n')

In [ ]:
recv_msg = ""
recv_msg += target.read() #you might have to run this block a few times to get the full message
print(recv_msg)

In [ ]:
scope.arm()
target.simpleserial_write('p', msg)
ret = scope.capture()
trace = scope.get_last_trace()

In [ ]:
if ret:
    print("Capture timed out!")
else:
    print("Capture successful!")
    trace = scope.get_last_trace()
    print(f"Trace length: {len(trace)} samples")
    print(f"Trace range: {min(trace):.3f} to {max(trace):.3f}")

In [ ]:
# Using the convenience function
msg = bytearray([0x12, 0x34, 0x56, 0x78] + [0]*12)  # Some test data

trace = cw.capture_trace(scope, target, msg)

if trace is not None:
    print("Trace captured successfully!")
    print(f"Input data: {trace.textin}")
    print(f"Output data: {trace.textout}")
    print(f"Trace samples: {len(trace.wave)}")
else:
    print("Trace capture failed")